## AIHub Json Parsing

### Development Environment

In [1]:
%pip install kss==3.7.3

  Using cached kss-3.7.3-py3-none-any.whl
  Using cached emoji-1.2.0-py3-none-any.whl (131 kB)
  Using cached regex-2023.5.5-cp311-cp311-win_amd64.whl (267 kB)
  Using cached more_itertools-9.1.0-py3-none-any.whl (54 kB)
Note: you may need to restart the kernel to use updated packages.


KSS Argument Error: Restart Jupyter Kernel Runtime

In [2]:
%pip install python-mecab-ko

  Using cached python_mecab_ko-1.3.3-cp311-cp311-win_amd64.whl (814 kB)
  Using cached python_mecab_ko_dic-2.1.1.post2-py3-none-any.whl (34.5 MB)
Note: you may need to restart the kernel to use updated packages.


KSS 3.7.3 matches python-mecab-ko

In [3]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import re
import os
import kss
import json
from mecab import MeCab
import pandas as pd
from glob import glob

In [2]:
pwd

'D:\\AIHUB'

### Function

In [3]:
def json_file_name_list(path_list):
    for i in path_list:
        file_name = glob(i, recursive = True)
    return file_name

In [4]:
def train_valid_json_file_name_list(path_list):
    for i in path_list:
        if 'rain' in i:
            train_file_name = glob(i, recursive = True)
        elif 'alid' in i:  
            valid_file_name = glob(i, recursive = True)
    return train_file_name, valid_file_name

In [5]:
def divide_source_file_list(l, n): 
    for i in range(0, len(l), n): 
        yield l[i:i + n] 

In [6]:
def txt_file_name_list(source_file_nested_list, folder_corpus_type_name):
   
  text_file_name_list = []

  for i in range(len(source_file_nested_list)):
    txt_file_name = folder_corpus_type_name + str(i) + ".txt"
    text_file_name_list.append(txt_file_name)

  return text_file_name_list

In [7]:
def train_valid_txt_file_name_list(source_file_nested_list, folder_corpus_type_name):
   
  train_valid_set = source_file_nested_list[0][0]
  text_file_name_list = []

  if 'rain' in train_valid_set:
    for i in range(len(source_file_nested_list)):
      txt_file_name = folder_corpus_type_name + str(i) + ".txt"
      text_file_name_list.append(txt_file_name)

  elif 'alid' in train_valid_set:
    for i in range(len(source_file_nested_list)):
      txt_file_name = folder_corpus_type_name + str(i) + ".txt"
      text_file_name_list.append(txt_file_name)  

  return text_file_name_list

In [23]:
def post_txt_file_name_list(corpus_list):
   
  post_corpus_list = []

  for corpus_file in corpus_list:
    post_corpus = corpus_file.replace("pro", "post")
    post_corpus_list.append(post_corpus)

  return post_corpus_list

In [8]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            sentence = re.sub(r"\[.*?\]|\{.*?\}", "",  sentence)
            # The String starts with a letter
            # The String ends with [. ! ?]
            # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                if sentence[-1] == ".":
                    preprocessing_sentence_list.append(sentence)
                else:
                    preprocessing_sentence_list.append(sentence + ".")
            # 문장의 끝에 '.' 추가

    return preprocessing_sentence_list

In [9]:
def informal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거
    for sentence in kss.split_sentences(source, use_heuristic=True,
                                        num_workers=32):
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            sentence = re.sub(r"\[.*?\]|\{.*?\}", "",  sentence)
            # The String starts with a letter
            # The String ends with [. ! ?]
            # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                if sentence[-1] == ".":
                    preprocessing_sentence_list.append(sentence)
                else:
                    preprocessing_sentence_list.append(sentence + ".")
            # 문장의 끝에 '.' 추가
   
    return preprocessing_sentence_list

### AIHUB 도서자료 기계독해

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=92)

In [35]:
path_list = ['AIHUB_도서자료 기계독해/'+ '/**/*.json']
file_name = json_file_name_list(path_list)

In [36]:
the_number_of_file = len(file_name) 
print("The number of file:", the_number_of_file)
n = 1
print("The number of list element:", n)
file_nested_list = list(divide_source_file_list(file_name, n))

The number of file: 2
The number of list element: 1


In [58]:
text_file_name_list = txt_file_name_list(file_nested_list,
                                         "demo/test_pro/AIHUB_test_")
the_numer_of_txt_file = len(text_file_name_list)
print("The number of txt file:", the_numer_of_txt_file)

The number of txt file: 2


In [38]:
def list_length_checker(source_file_nested_list, batch_size):
    
    the_number_of_total_txt_file = 0
    the_number_of_txt_file_list = []
    
    for source_file_list in source_file_nested_list:   
        for i in range(len(source_file_list)):
            sentence_list = []
            
            if type(source_file_list) == str:
                source_file = source_file_list

            elif type(source_file_list) != str:
                source_file = source_file_list[i]
            

            with open(source_file, 'r', encoding='utf-8') as one_json_file:
                one_json_sample = json.load(one_json_file) 

            source_df = pd.DataFrame(one_json_sample['data'])
            source_dict = dict(source_df['paragraphs'].explode())
            source_json = pd.json_normalize(source_dict)
            source_list = list(source_json.filter(regex='context').values[0])

            source_filter_02 = source_json.filter(regex='[0123456789]').values[0]
            source_df_02 = pd.DataFrame(source_filter_02)
            
            for source_df_02_value in source_df_02.values:
              for value in source_df_02_value[0]:
                try:
                    source_list.append(value['context'])
                except:
                    pass
                    
            the_number_of_txt_file = ((len(source_list) // batch_size) + 1)

            print("File:", source_file)    
            print("Index:", i, "  ", "Length of Source List:", len(source_list), \
                "  ", "The number of txt file:", the_number_of_txt_file, "\n")
            the_number_of_txt_file_list.append(the_number_of_txt_file)
            the_number_of_total_txt_file  += the_number_of_txt_file

    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)
    
    return the_number_of_total_txt_file, the_number_of_txt_file_list

In [55]:
def make_corpus_txt_with_batch_list(source_file_nested_list,
                                    text_file_name_list,
                                    batch_size, the_number_of_txt_file_list):

  print("[Size]")
  print("The numnber of preprocessing corpus: " + str(sum(the_number_of_txt_file_list)))
  print("\n[Order]")
  num = 0
  for i in range(len(source_file_nested_list)):
    source_file_list = source_file_nested_list[i]
    
    for j in range(len(source_file_list)):
      
      if type(source_file_list) == str:
        source_file = source_file_list

      elif type(source_file_list) != str:
        source_file = source_file_list[i]

      with open(source_file, 'r', encoding='utf-8') as one_json_file:
        one_json_sample = json.load(one_json_file)

      source_df = pd.DataFrame(one_json_sample['data'])
      source_dict = dict(source_df['paragraphs'].explode())
      source_json = pd.json_normalize(source_dict)
      source_list = list(source_json.filter(regex='context').values[0])
        
      source_filter_02 = source_json.filter(regex='[0123456789]').values[0]
      source_df_02 = pd.DataFrame(source_filter_02)
        
      for source_df_02_value in source_df_02.values:
        for value in source_df_02_value[0]:
          try:
              source_list.append(value['context'])
          except:
            pass

      n = batch_size
      source_batch_list = list(divide_source_file_list(source_list, n))
        
      for source_list in source_batch_list:   
          
        num += 1
        print(str(num), end=" ")  
        
        with open(os.path.join('AIHUB_corpus/' + text_file_name_list[i][:-4] + "_" + str(num) + ".txt"), "a", encoding='utf-8') as fp:        
            fp.write("\n".join(source_list))         

In [56]:
batch_size = 100
the_number_of_txt_file, the_number_of_txt_file_list = list_length_checker(file_nested_list, batch_size)

File: AIHUB_도서자료 기계독해\Training\도서_220419_add\도서_220419_add.json
Index: 0    Length of Source List: 225000    The number of txt file: 2251 

File: AIHUB_도서자료 기계독해\Validation\도서.json
Index: 0    Length of Source List: 12500    The number of txt file: 126 

Batch Size: 100
The number of txt file: 2377


In [59]:
batch_size = 100
make_corpus_txt_with_batch_list(file_nested_list[:1], text_file_name_list[:1],
                batch_size, the_number_of_txt_file_list[:1])

[Size]
The numnber of preprocessing corpus: 2251

[Order]


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

2217 2218 2219 2220 2221 2222 2223 2224 2225 2226 2227 2228 2229 2230 2231 2232 2233 2234 2235 2236 2237 2238 2239 2240 2241 2242 2243 2244 2245 2246 2247 2248 2249 2250 

In [ ]:
batch_size = 100
make_corpus_txt_with_batch_list(file_nested_list[1:], text_file_name_list[1:],
                batch_size, the_number_of_txt_file_list[1:])

In [24]:
# before_total_corpus_list = glob("AIHUB_corpus/demo/machine_reading_before/AIHUB_machine_reading_train_" +"*.txt")
# before_total_corpus_list += glob("AIHUB_corpus/demo/machine_reading_before/AIHUB_machine_reading_valid_" +"*.txt")
pro_total_corpus_list = glob("AIHUB_corpus/demo/test_pro/AIHUB_test_" +"*.txt")

In [26]:
len(pro_total_corpus_list)

2250

In [27]:
post_total_corpus_list = post_txt_file_name_list(pro_total_corpus_list)

In [31]:
line_list = []
line_num = 0
with open(pro_total_corpus_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 1:
           line_list.append(line)
for line in line_list:
    print(line, end="\n\n")

교사로 임용된 후 약 5년까지의 교사를 초임교사라고 하며, 이 기간 동안 길러지는 능력과 태도 등의 습성은 교사로 재직하는 나머지 기간 동안 교사의 교육활동 전반에 결정적인 영향을 준다(남유섭, 2012; 이현숙, 2004). 또한 교사의 생애사적 관점에서 초임교사를 연구한 임국화(2012)는 초임교사 시절의 경험은 교직생애 발달에 중요한 영향을 미치기 때문에 매우 중요한 시기라고 주장하였다. 이처럼 전문가로서의 교직 생활에 지대한 영향을 끼치는 초임교사 시기에 초임교사가 평가 본연의 특성과 평가활동이 지니는 교육적 영향력을 제대로 인식한다면, 초임교사 개인의 전문성 발달은 물론 향후 교직에 있는 동안 학생을 평가하는 평가 전문성에서도 긍정적인 변화가 있을 것으로 생각한다. 그러므로 초등학교 초임교사의 학생평가 전문성 수준을 분석하여 교직의 학생평가 전문성 함양에 기여할 수 있는 기초적인 정보를 제공하는 일은 의의가 있다고 할 수 있다. 



In [33]:
line_list = []
line_num = 0
with open(pro_total_corpus_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    for line in lines:
        line_num += 1
        if line_num <= 1:  
            sentences = formal_preprocessing_text(line)
            for sentence in sentences:
                line_list.append(sentence) 
            
for line in line_list:
    print(line, end="\n\n")

교사로 임용된 후 약 5년까지의 교사를 초임교사라고 하며  이 기간 동안 길러지는 능력과 태도 등의 습성은 교사로 재직하는 나머지 기간 동안 교사의 교육활동 전반에 결정적인 영향을 준다(남유섭  2012  이현숙  2004)  또한 교사의 생애사적 관점에서 초임교사를 연구한 임국화(2012)는 초임교사 시절의 경험은 교직생애 발달에 중요한 영향을 미치기 때문에 매우 중요한 시기라고 주장하였다.

이처럼 전문가로서의 교직 생활에 지대한 영향을 끼치는 초임교사 시기에 초임교사가 평가 본연의 특성과 평가활동이 지니는 교육적 영향력을 제대로 인식한다면  초임교사 개인의 전문성 발달은 물론 향후 교직에 있는 동안 학생을 평가하는 평가 전문성에서도 긍정적인 변화가 있을 것으로 생각한다.

그러므로 초등학교 초임교사의 학생평가 전문성 수준을 분석하여 교직의 학생평가 전문성 함양에 기여할 수 있는 기초적인 정보를 제공하는 일은 의의가 있다고 할 수 있다.



In [34]:
print("[Size]")
print("The numnber of preprocessing corpus: " + str(len(pro_total_corpus_list)))
print("\n[Order]")
num = 0
    
for pro, post in zip(pro_total_corpus_list, post_total_corpus_list):
    
    sentence_list = []
    with open(pro, 'r', encoding='utf-8') as f:
        lines = f.read().splitlines() 
        for line in lines:
            sentences = formal_preprocessing_text(line)
            for sentence in sentences:
                sentence_list.append(sentence) 

    num += 1
    print(str(num), end=" ")  
        
    with open(post, 'a', encoding='utf-8') as fp:
        fp.write("\n".join(sentence_list))

KeyboardInterrupt: 

In [ ]:
corpus_list = glob("AIHUB_corpus/demo/reading_books_by_machine/AIHUB_reading_books_by_machine_" +"*.txt")

In [ ]:
with open('AIHUB_corpus/demo/AIHUB_reading_books_by_machine.txt', 'w') as f:
    for corpus in corpus_list:
        with open(corpus) as text:
            for line in text:
                f.write(line)